# Libs

In [35]:
import gym
import pandas as pd

from evaluate import *

# Simulation Setup

In [4]:
input_dir = './agent'
output_dir  = './out' 
sim_cfg = './cfg/simulator.cfg'

In [10]:
simulator_cfg_file = sim_cfg
submission_dir, scores_dir = resolve_dirs(
    os.path.dirname("evaluate.py"), input_dir, output_dir
)
agent_spec,gym_cfg = load_agent_submission(submission_dir)

INFO:/starter-kit/evaluate.py:root_path=[.git, .gitignore, .ipynb_checkpoints, agent, agent.zip, cfg, data, demo.py, evaluate.py, log, out, readme.md, ui, Untitled.ipynb, __pycache__]
INFO:/starter-kit/evaluate.py:input_dir=[agent.py, gym_cfg.py, __pycache__]
INFO:/starter-kit/evaluate.py:output_dir=[.gitignore, scores.json]
INFO:/starter-kit/evaluate.py:submission_dir=[agent.py, gym_cfg.py, __pycache__]
INFO:/starter-kit/evaluate.py:scores_dir=[.gitignore, scores.json]
INFO:/starter-kit/evaluate.py:files under submission dir:[agent.py, gym_cfg.py, __pycache__]


# Simulation Init

In [12]:
gym_configs = gym_cfg.cfg
simulator_configs = read_config(simulator_cfg_file)
env = gym.make(
    'CBEngine-v0',
    simulator_cfg_file=simulator_cfg_file,
    thread_num=1,
    gym_dict = gym_configs
)

In [17]:
observations, infos = env.reset()
agent_id_list = []
for k in observations:
    agent_id_list.append(int(k.split('_')[0]))
agent_id_list = list(set(agent_id_list))
agent = agent_spec['test']
agent.load_agent_list(agent_id_list)

# Run Simulation (Single Step)

In [61]:
actions = {}
all_info = {
    'observations':observations,
    'info':infos
}
actions = agent.act(all_info)

observations, rewards, dones, infos = env.step(actions)

Warnning: at step 7 , agent 42167350403 switch to phase 4 . Maybe empty road
Warnning: at step 7 , agent 42167350420 switch to phase 4 . Maybe empty road
Warnning: at step 7 , agent 14670355735 switch to phase 4 . Maybe empty road
Warnning: at step 7 , agent 42381408549 switch to phase 4 . Maybe empty road
Warnning: at step 7 , agent 14790105773 switch to phase 4 . Maybe empty road
Warnning: at step 7 , agent 42167350476 switch to phase 4 . Maybe empty road


# Process Observation

In [62]:
# preprocess observations
observations_for_agent = {}
for key,val in observations.items():
    observations_agent_id = int(key.split('_')[0])
    observations_feature = key[key.find('_')+1:]
    if(observations_agent_id not in observations_for_agent.keys()):
        observations_for_agent[observations_agent_id] = {}
    observations_for_agent[observations_agent_id][observations_feature] = val

In [63]:
obs = []
for agent_id, agent_obs in observations_for_agent.items():
    sim_step = agent_obs['lane_speed'][0]
    lane_speed = agent_obs['lane_speed'][1:]
    lane_vehicle_num = agent_obs['lane_vehicle_num'][1:]
    assert len(lane_speed) == len(lane_vehicle_num)
    for idx, speed in enumerate(lane_speed):
        road_idx = idx // 3
        lane_idx = idx % 3
        veh_num = lane_vehicle_num[idx]
        obs.append([sim_step, agent_id, road_idx, lane_idx, speed, veh_num])

obs_df = pd.DataFrame(obs, columns=['sim_step', 'agent_id', 'road_idx', 'lane_idx', 'speed', 'veh_num'])
obs_df = obs_df[(obs_df.speed != -1) & (obs_df.veh_num != -1)].reset_index(drop=True)

# Process Roadnet

In [69]:
def process_roadnet(roadnet_file):
    # intersections[key_id] = {
    #     'have_signal': bool,
    #     'end_roads': list of road_id. Roads that end at this intersection. The order is random.
    #     'start_roads': list of road_id. Roads that start at this intersection. The order is random.
    #     'lanes': list, contains the lane_id in. The order is explained in Docs.
    # }
    # roads[road_id] = {
    #     'start_inter':int. Start intersection_id.
    #     'end_inter':int. End intersection_id.
    #     'length': float. Road length.
    #     'speed_limit': float. Road speed limit.
    #     'num_lanes': int. Number of lanes in this road.
    #     'inverse_road':  Road_id of inverse_road.
    #     'lanes': dict. roads['lanes'][lane_id] = list of 3 int value. Contains the Steerability of lanes
    # }
    # agents[agent_id] = list of length 8. contains the inroad0_id, inroad1_id, inroad2_id,inroad3_id, outroad0_id, outroad1_id, outroad2_id, outroad3_id

    intersections = {}
    roads = {}
    agents = {}

    agent_num = 0
    road_num = 0
    signal_num = 0
    with open(roadnet_file, 'r') as f:
        lines = f.readlines()
        cnt = 0
        pre_road = 0
        is_obverse = 0
        for line in lines:
            line = line.rstrip('\n').split(' ')
            if ('' in line):
                line.remove('')
            if (len(line) == 1):
                if (cnt == 0):
                    agent_num = int(line[0])
                    cnt += 1
                elif (cnt == 1):
                    road_num = int(line[0]) * 2
                    cnt += 1
                elif (cnt == 2):
                    signal_num = int(line[0])
                    cnt += 1
            else:
                if (cnt == 1):
                    intersections[int(line[2])] = {
                        'have_signal': int(line[3]),
                        'end_roads': [],
                        'start_roads': [],
                        'lanes':[]
                    }
                elif (cnt == 2):
                    if (len(line) != 8):
                        road_id = pre_road[is_obverse]
                        roads[road_id]['lanes'] = {}
                        for i in range(roads[road_id]['num_lanes']):
                            roads[road_id]['lanes'][road_id * 100 + i] = list(map(int, line[i * 3:i * 3 + 3]))
                        is_obverse ^= 1
                    else:
                        roads[int(line[-2])] = {
                            'start_inter': int(line[0]),
                            'end_inter': int(line[1]),
                            'length': float(line[2]),
                            'speed_limit': float(line[3]),
                            'num_lanes': int(line[4]),
                            'inverse_road': int(line[-1])
                        }
                        roads[int(line[-1])] = {
                            'start_inter': int(line[1]),
                            'end_inter': int(line[0]),
                            'length': float(line[2]),
                            'speed_limit': float(line[3]),
                            'num_lanes': int(line[5]),
                            'inverse_road': int(line[-2])
                        }
                        intersections[int(line[0])]['end_roads'].append(int(line[-1]))
                        intersections[int(line[1])]['end_roads'].append(int(line[-2]))
                        intersections[int(line[0])]['start_roads'].append(int(line[-2]))
                        intersections[int(line[1])]['start_roads'].append(int(line[-1]))
                        pre_road = (int(line[-2]), int(line[-1]))
                else:
                    # 4 out-roads
                    signal_road_order = list(map(int, line[1:]))
                    now_agent = int(line[0])
                    in_roads = []
                    for road in signal_road_order:
                        if (road != -1):
                            in_roads.append(roads[road]['inverse_road'])
                        else:
                            in_roads.append(-1)
                    in_roads += signal_road_order
                    agents[now_agent] = in_roads
    for agent, agent_roads in agents.items():
        intersections[agent]['lanes'] = []
        for road in agent_roads:
            ## here we treat road -1 have 3 lanes
            if (road == -1):
                for i in range(3):
                    intersections[agent]['lanes'].append(-1)
            else:
                for lane in roads[road]['lanes'].keys():
                    intersections[agent]['lanes'].append(lane)

    return intersections, roads, agents

In [70]:
intersections, roads, agents = process_roadnet('./data/roadnet_warm_up.txt')

inters_df = pd.DataFrame(intersections).T.reset_index()
inters_df.rename({"index":"agent_id"}, axis=1, inplace=True)

roads_df = pd.DataFrame(roads).T.reset_index()
roads_df.rename({"index":"road_id"}, axis=1, inplace=True)

# Define Pressure

In [68]:
obs_df[(obs_df.agent_id == 14670355735) & (obs_df.lane_idx < 2)]

,sim_step,agent_id,road_idx,lane_idx,speed,veh_num
0,8,14670355735,0,0,-2,0
1,8,14670355735,0,1,-2,0
3,8,14670355735,1,0,-2,0
4,8,14670355735,1,1,-2,0
6,8,14670355735,2,0,-2,0
7,8,14670355735,2,1,-2,0
9,8,14670355735,4,0,-2,0
10,8,14670355735,4,1,-2,0
12,8,14670355735,5,0,-2,0
13,8,14670355735,5,1,-2,0
